# Networks: structure, evolution & processes
**Internet Analytics - Lab 2**

---

**Group:** W

**Names:**

* Olivier Cloux
* Thibault Urien
* Saskia Reiss

---

#### Instructions

*This is a template for part 4 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

---

## 2.4 PageRank

### 2.4.1 Random Surfer Model

#### Exercise 2.12

In [ ]:
#necessary imports
import networkx as nx
import matplotlib.pyplot as plt
import random as random
import csv

#global variables
jumps = 1000
dangling_fac = 0.15

In [ ]:
#Load graphs
G1=nx.read_adjlist('../data/components.graph', create_using=nx.DiGraph(), nodetype=int)
G2=nx.read_adjlist('../data/absorbing.graph', create_using=nx.DiGraph(), nodetype=int)
Gwiki=nx.read_adjlist('../data/wikipedia.graph', create_using=nx.DiGraph(), nodetype=int)


In [ ]:
#helper functions, to make code cleaner
def print_dict_sorted(d, precision):
    """print a dictionnary sorted by it's keys
    argument 'precision': gives number of desired leading zeros
    """
    print("Weight of each node :")
    for k, v in sorted(d.items()): 
        print("Node",str(k).zfill(precision),"has score", v)

#Surfer
def surfer(G, jumps):
    """Surfs through an networkX graph"""
    nodes_list = G.nodes()
    nodes_and_weight = dict(zip(G.nodes(), [0]*G.number_of_nodes()))
    seed = random.sample(nodes_list, 1).pop()
    current = seed
    i = 0
    while i < jumps:
        
        i+= 1
        nodes_and_weight[current] += 1
        possible_nodes = G.edges(current)
        if len(possible_nodes) >= 1: #check for dead end
            current = random.sample(possible_nodes,1).pop()[1]
        else:
            print("Reached a dead end after",i,"jumps and",i+1,"visited pages. No links in this page")
            break
    
    #return normalized version
    nodes_and_weight.update((k, v/i) for k,v in nodes_and_weight.items())
    return nodes_and_weight

#### Results of components graph
We see below that not all components are connected (the network is not one giant component). Thus, entering at one node traps us in the connected component and excludes us from different component(s). This behaviour is to be avoided.

In [ ]:
surf1 = surfer(G1, jumps)
print_dict_sorted(surf1, 1)    

#### Results of absorbing graph
The result here is better seen when launching the code multiple times.
We quickly see there is a dangling node (node with no outgoing edges). This denotes an absorbing behaviour, meaning once this node is reached we can't keep crawling.

In [ ]:
surf2 = surfer(G2, jumps)
print_dict_sorted(surf2, 1)

#### Exercise 2.13

In [ ]:
def modified_surfer(G, jumps, dang_fac):
    """Surfs through an networkX graph"""
    nodes_list = G.nodes()
    nodes_and_weight = dict(zip(G.nodes(), [0]*G.number_of_nodes())) #creates dict of node ID and it's score (0)
    seed = random.sample(nodes_list, 1).pop()
    current = seed
    i = 0
    while i < jumps:
        
        nodes_and_weight[current] += 1
        possible_nodes = G.edges(current)
        if len(possible_nodes) == 0 or random.randrange(0, 1) < dang_fac: 
            current = random.sample(nodes_list, 1).pop() #take one node at random
        else:
            current = random.sample(possible_nodes,1).pop()[1] #pick one in linked nodes
        i += 1
        
    #return normalized version
    nodes_and_weight.update((k, v/i) for k,v in nodes_and_weight.items())
    return nodes_and_weight

#### Results of components graph with modified surfer
The below result seems much better, as we now visited all components. 

In [ ]:
surf1 = modified_surfer(G1, jumps, dangling_fac)
print_dict_sorted(surf1, 1)

#### Results of absorbing graph with modified surfer
Our surfer does not halt anymore when reaching a dangling node, which is the correct behaviour. 

In [ ]:
surf2 = modified_surfer(G2, jumps, dangling_fac)
print_dict_sorted(surf2, 1)

---

### 2.4.2 Power Iteration Method

#### Exercise 2.14: Power Iteration method

In [ ]:
import numpy as np
np.set_printoptions(threshold=np.inf)
theta = 0.85

def google_matrix(graph):
    #Compute the google matrix for the given graph
    N = graph.number_of_nodes()
    w = np.zeros(N) #dangling indicator
    H = np.zeros((N, N))

    for node in graph.nodes(): #analyse every node iteratively
        edges = (graph.edges(node)) #type : edges = list of connected nodes
        if(len(edges) == 0): #no outgoing egde <-> dangling node
            w[node] = 1
        else :
            edges_indices = np.array([x[1] for x in edges], dtype=int)  
            H[node][edges_indices] = 1/len(edges)
    H2 = H + ((w) * np.ones((N,1)).T)/N
    G = theta*H2 + (1-theta)*np.ones((N, N))/N
    return G

In [ ]:
G = google_matrix(Gwiki)
N = Gwiki.number_of_nodes()
pivec = np.ones(N)/N #original pi
for i in range(5000): #long operation, decrease for faster result
    pivec = pivec @ G

In [ ]:
with open('../data/wikipedia_titles.tsv', newline='\n') as datafile:
    fin = datafile.read().splitlines(True)[1:]
    reader = csv.reader(fin, delimiter='\t')
dictionnary = dict((int(row[0]), row[1]) for row in reader)

In [ ]:
desired_max = 10
# np.argpartition is an efficient way to get top N values but returns them unsorted.
max_indices = np.argpartition(pivec, -desired_max)[-desired_max:]
value_to_index = map(lambda x : (pivec[x], x), max_indices)
#Sort only the top N values.
sorted_indices = sorted(value_to_index, key= lambda x:x[0], reverse=True)

print("The",desired_max,"max elements are :")
j = 1
for i in sorted_indices:    
    print("#",j,":",dictionnary[i[1]],"with score",i[0])
    j+=1

---

### 2.4.3 Gaming the system *(Bonus)*

#### Exercise 2.15 *(Bonus)*

In [ ]:
id_history = list(dictionnary.keys())[list(dictionnary.values()).index('History of mathematics')]
desired_max = 300
max_indices = np.argpartition(pivec, -desired_max)[-desired_max:]
#The new links between the top N page and the page we try to boost.
mapped = zip(max_indices, [id_history]*desired_max)
print(pivec[id_history])
Gwiki2 = nx.read_adjlist('../data/wikipedia.graph', create_using=nx.DiGraph(), nodetype=int)

Gwiki2.add_edges_from(mapped)

G2 = google_matrix(Gwiki2)
pivec2 = np.ones(N)/N #original pi
for i in range(5000): #long operation, decrease for faster result
    pivec2 = pivec2 @ G2
    
print(pivec2[id_history])

